### Importing libraries to use

In [2]:

import pandas as pd
import re  


### Uploading dataset to work out

In [3]:
data = pd.read_csv('data.csv')  
data.head()


,ESTU_GENERO,PERIODO,ESTU_PAIS_RESIDE,ESTU_DEPTO_RESIDE,ESTU_MCPIO_RESIDE,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_ESTRATOVIVIENDA,...,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_CUANTOSCOMPARTEBAÑO,ESTU_HORASSEMANATRABAJA,INST_NOMBRE_INSTITUCION,ESTU_PRGM_ACADEMICO,PUNT_GLOBAL,Unnamed: 23,CANT_DATOS
0,M,2020,COLOMBIA,BOGOTÁ,BOGOTÁ D.C.,Entre 1 millón y menos de 2.5 millones,Si,Secundaria (Bachillerato) incompleta,Primaria incompleta,Estrato 2,...,No,No,No,NINGUNA,Más de 30 horas,CORPORACION UNIVERSITARIA MINUTO DE DIOS -UNIM...,ADMINISTRACIÓN FINANCIERA,117,NaN,23.0
1,M,2020,VENEZUELA,BOGOTÁ,BOGOTÁ D.C.,Entre 4 millones y menos de 5.5 millones,No,Educación profesional completa,Educación profesional completa,Estrato 4,...,Si,No,Si,1,Más de 30 horas,UNIVERSIDAD DE SAN BUENAVENTURA-BOGOTÁ D.C.,ADMINISTRACION DE EMPRESAS,164,NaN,23.0
2,F,2020,COLOMBIA,CESAR,BOSCONIA,Entre 500 mil y menos de 1 millón,No,Secundaria (Bachillerato) completa,Primaria incompleta,Estrato 2,...,No,No,No,3 o 4,Entre 11 y 20 horas,UNIVERSIDAD DEL MAGDALENA - UNIMAGDALENA-SANTA...,ADMINISTRACIÓN DE LA SEGURIDAD Y SALUD EN EL...,124,NaN,23.0
3,F,2020,COLOMBIA,LA GUAJIRA,MAICAO,Entre 500 mil y menos de 1 millón,Si,Primaria incompleta,Ninguno,Estrato 1,...,No,No,No,MAS DE 6,Entre 21 y 30 horas,UNIVERSIDAD DE LA GUAJIRA-RIOHACHA,LICENCIATURA EN ETNOEDUCACION,133,NaN,23.0
4,F,2020,COLOMBIA,ATLANTICO,BARRANQUILLA,Entre 500 mil y menos de 1 millón,Si,Primaria completa,Primaria completa,Estrato 1,...,No,No,No,5 o 6,Más de 30 horas,CORPORACION UNIVERSITARIA MINUTO DE DIOS -UNIM...,LICENCIATURA EN PEDAGOGÍA INFANTIL,117,NaN,23.0


### Scoping dataset to 2022-23 engineer popuplation from Bogota DC

In [4]:

degrees = []
for degree in data['ESTU_PRGM_ACADEMICO'].unique():
    if re.match("INGENIERIA", degree):
        degrees.append(degree)

data = data[data['ESTU_PRGM_ACADEMICO'].isin(degrees)]
data = data[data['PERIODO'].isin([2022,2023])]
data = data.drop('PERIODO', axis=1)
data = data.drop('ESTU_MCPIO_RESIDE', axis=1)
data = data.drop('ESTU_DEPTO_RESIDE', axis=1)
data = data.drop('ESTU_PAIS_RESIDE', axis=1)
data = data[data['ESTU_PRGM_ACADEMICO'].isin(degrees)]
data = data.drop('ESTU_PRGM_ACADEMICO', axis=1)
data = data.drop('INST_NOMBRE_INSTITUCION', axis=1)
data = data.drop('Unnamed: 23', axis=1)
data = data.drop('CANT_DATOS', axis=1)



### Cleaning from duplicates and Nan values to the dataset

In [5]:
data = data.drop_duplicates()
data = data.dropna(how='any')


### Transforming data

In [6]:

boolean_variables = ['ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENELAVADORA', 'FAMI_TIENEHORNOMICROOGAS',
                      'FAMI_TIENESERVICIOTV', 'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENEMOTOCICLETA', 'FAMI_TIENECONSOLAVIDEOJUEGOS']

for variable in boolean_variables:

  data[variable] = data[variable].replace({'Si': 1, 'No': 0})


/tmp/ipykernel_2551/4208731398.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[variable] = data[variable].replace({'Si': 1, 'No': 0})


### Transforming categorical variables

In [7]:


data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({'Entre 4 millones y menos de 5.5 millones': 4.75, 'Entre 2.5 millones y menos de 4 millones': 3.25, 
                                                                                          'Entre 5.5 millones y menos de 7 millones': 6.25, 'No pagó matrícula': 0, 
                                                                                          'Entre 1 millón y menos de 2.5 millones': 1.75, 'Menos de 500 mil': 0.5, 
                                                                                          'Más de 7 millones' : 7, 'Entre 500 mil y menos de 1 millón' : 0.75 })

data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].replace({'Secundaria (Bachillerato) completa' : 4, 'Primaria incompleta' : 1, 'Educación profesional completa' : 8,
                                                                  'Secundaria (Bachillerato) incompleta' : 3, 'Técnica o tecnológica completa' : 6, 'Primaria completa' : 2, 'Postgrado' : 9,
                                                                  'Educación profesional incompleta' : 7, 'Técnica o tecnológica incompleta' : 5, 'No sabe' : 0, 'Ninguno' : 0, 'No Aplica' : 0})

data['FAMI_EDUCACIONMADRE'] = data['FAMI_EDUCACIONMADRE'].replace({'Secundaria (Bachillerato) completa' : 4, 'Primaria incompleta' : 1, 'Educación profesional completa' : 8,
                                                                  'Secundaria (Bachillerato) incompleta' : 3, 'Técnica o tecnológica completa' : 6, 'Primaria completa' : 2, 'Postgrado' : 9,
                                                                  'Educación profesional incompleta' : 7, 'Técnica o tecnológica incompleta' : 5, 'No sabe' : 0, 'Ninguno' : 0, 'No Aplica' : 0})

data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].replace({'Estrato 3' : 3, 'Estrato 2' : 2, 'Estrato 4' : 4, 'Estrato 1' : 1, 'Estrato 5' : 5,
                                                                      'Sin Estrato' : 0, 'Estrato 6' : 6})

data['FAMI_CUANTOSCOMPARTEBAÑO'] = data['FAMI_CUANTOSCOMPARTEBAÑO'].replace({'NINGUNA' : 0, '5 o 6' : 5.5, '3 o 4' : 3.5, '1' : 1, 'MAS DE 6' : 6, '2' : 2})

data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].replace({'Más de 30 horas' : 30, 'Entre 11 y 20 horas' : 15.5, 'Entre 21 y 30 horas' : 25.5,

                                                                            'Menos de 10 horas' : 10, '0' : 0})
data

/tmp/ipykernel_2551/272576009.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({'Entre 4 millones y menos de 5.5 millones': 4.75, 'Entre 2.5 millones y menos de 4 millones': 3.25,
/tmp/ipykernel_2551/272576009.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].replace({'Secundaria (Bachillerato) completa' : 4, 'Primaria incompleta' : 1, 'Educación profesional completa' : 8,
/tmp/ipyk

,ESTU_GENERO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENESERVICIOTV,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_CUANTOSCOMPARTEBAÑO,ESTU_HORASSEMANATRABAJA,PUNT_GLOBAL
445200,F,6.25,0,6,8,2,1,0,1,0,1,0,0,0,3.5,15.5,165
445201,M,6.25,1,5,6,3,1,0,1,1,0,0,0,0,3.5,25.5,138
445205,M,4.75,0,4,6,2,0,1,1,0,1,0,0,0,3.5,15.5,129
445209,F,4.75,0,1,6,1,1,1,1,0,1,0,0,0,3.5,10.0,164
445213,M,3.25,1,6,7,3,1,1,1,0,1,1,1,0,2.0,15.5,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663052,F,0.00,0,1,1,3,1,1,1,1,1,0,1,1,3.5,10.0,121
663078,M,0.00,0,9,0,3,1,1,1,0,1,1,0,0,3.5,30.0,135
663105,M,6.25,0,6,9,3,1,1,1,1,1,1,0,0,3.5,30.0,129
663110,M,0.00,1,0,1,1,1,1,1,0,0,0,1,0,3.5,30.0,125


In [8]:
data.to_csv('data_cleaned.csv', index=False)

In [9]:
#possible solutions
#for i, k in enumerate(data['FAMI_EDUCACIONPADRE'].unique()):
#import re
#pattern = r'\b\d+\b' 
#def get_nums(string : str) -> list :
#  return re.findall(pattern, string)[0]

# xd = lambda y: int(get_nums(y)[0])
# xd('34')
